In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import scipy.sparse
import time
import itertools
import sys
import pickle
import helper

tags2int = {"TAG": 0, "shooterName":1, "killedNum":2, "woundedNum":3, "city":4}
int2tags = ["TAG",'shooterName','killedNum','woundedNum','city']
tags = [0,1,2,3,4]


def get_word_vocab(data, prune):
    num_words = 0
    word_vocab = {}
    for sentence in data:
        words_in_sentence = set()
        for word in sentence[0]:
            if word.lower() in words_in_sentence:
                continue
            if word.lower() not in word_vocab:
                word_vocab[word.lower()] = 1
            else:
                word_vocab[word.lower()] += 1
            words_in_sentence.add(word.lower())
        num_words += len(sentence[0])
    feature_list = []
    prune_features(word_vocab,feature_list, prune)
    return num_words,word_vocab

# reduce dimensions by removing features that don't appear often (not used)
def prune_features(feature_vocab, featureList, prune):
    for w in feature_vocab.keys():
        if feature_vocab[w] <= prune:
            feature_vocab.pop(w,None)
    index = 0
    for w in feature_vocab.keys():
        feature_vocab[w] = index
        featureList.append(w)
        index += 1

# get feature matrix given a list of sentences and tags (used for training) n: previous n words
def get_feature_matrix_n(previous_n,next_n,data, num_words, word_vocab, other_features,first_n=10):
    num_features = len(word_vocab) + len(other_features)
    total_features = (previous_n+next_n+1)*num_features + len(word_vocab) + previous_n * len(tags) + first_n
    #print num_words, num_features, total_features
    dataY = np.zeros(num_words)
    dataX = scipy.sparse.lil_matrix((num_words, total_features))
    curr_word = 0
    for sentence in data:
        other_words_lower = set([s.lower for s in sentence[0]])
        for i in range(len(sentence[0])):
            word = sentence[0][i]
            word_lower = word.lower()
            if word_lower in word_vocab:
                dataX[curr_word,word_vocab[word_lower]] = 1
                for j in range(previous_n):
                    if i+j+1<len(sentence[0]):
                        dataX[curr_word+j+1,(j+1)*num_features+word_vocab[word_lower]] = 1
                for j in range(next_n): 
                    if i-j-1 >= 0:
                        dataX[curr_word-j-1,(previous_n+j+1)*num_features+word_vocab[word_lower]] = 1
            for (index, feature_func) in enumerate(other_features):
                if feature_func(word):
                    dataX[curr_word,len(word_vocab)+index] = 1
                    for j in range(previous_n):
                        if i + j + 1 < len(sentence[0]):
                            dataX[curr_word+j+1,(j+1)*num_features+len(word_vocab)+index] = 1
                    for j in range(next_n):
                        if i - j - 1 >= 0:
                            dataX[curr_word-j-1,(previous_n+j+1)*num_features+len(word_vocab)+index] = 1
            for other_word_lower in other_words_lower:
                if other_word_lower != word_lower and other_word_lower in word_vocab:
                    dataX[curr_word,(previous_n+next_n+1)*num_features + word_vocab[other_word_lower]] = 1
            for j in range(previous_n):
                if j < i:
                    dataX[curr_word,(previous_n+next_n+1)*num_features+len(word_vocab)+len(tags) * j + dataY[curr_word-j-1]] = 1
            if i < first_n:
                dataX[curr_word,(previous_n+next_n+1)*num_features + len(word_vocab) + previous_n * len(tags)+i] = 1
            dataY[curr_word] = sentence[1][i]
            curr_word += 1
    return dataX, dataY

# split sentence into a list of words and a list of tags
def separate_word_tag(sentence):
    parts = sentence.split()
    words = []
    tags = []
    i = 0
    for part in parts:
        i+=1
        #if i > 20:
        #    break
        words.append(part.split("_")[0])
        tags.append(tags2int[part.split("_")[1]])
    return [words,tags]

# return a list of raw sentences (unprocessed)
def load_data(filename):
    sentence_list = [line.rstrip('\n') for line in open(filename)][1::2]
    identifier = [line.rstrip('\n') for line in open(filename)][::2]
    return map(separate_word_tag,sentence_list), identifier

# prints a list of top 10 features for each class
def getTopFeatures(clf, tags, featureList):
    A = np.copy(clf.coef_)
    for i in tags:
        print int2tags[i]
        #A[i] = map(abs, A[i])
        indices = np.argsort(A[i])[-20:][::-1]
        print indices
        for j in indices:
            print featureList[j]

def save_list_first_names(infile_path,outfile_path):
    l = set()
    with open(infile_path) as infile:
        for line in infile:
            l.add(line.split()[0].lower())
    print len(l)
    pickle.dump(l,open(outfile_path, "wb" ))





In [2]:
training_file = "../data/tagged_data/whole_text_full_city2/train.tag" #sys.argv[1]

all_data, all_identifier = load_data(training_file)


In [10]:
trained_model = "trained_model3.p" #sys.argv[2]
previous_n = 0 #sys.argv[3]
next_n = 4
c = 10
prune = 5

# main loop
helper.load_constants()
all_data, all_identifier = load_data(training_file)
train_split = .6
split_index = int(len(all_data)*train_split)
train_data, train_identifier = all_data[:split_index], all_identifier[:split_index]
test_data, test_identifier = all_data[split_index:], all_identifier[split_index:]


## extract features
tic = time.clock()
num_words_train, word_vocab_train = get_word_vocab(train_data, prune)
num_words_test, word_vocab_test = get_word_vocab(test_data, prune)
num_words_all, word_vocab_all = get_word_vocab(all_data, prune)



trainX, trainY = get_feature_matrix_n(previous_n,next_n, train_data, num_words_train, word_vocab_all, helper.other_features)
testX, testY = get_feature_matrix_n(previous_n, next_n,   test_data, num_words_test, word_vocab_all, helper.other_features)

print time.clock()-tic

## train LR
print("training")
tic = time.clock()
clf = LogisticRegression(C=c, multi_class='multinomial', solver='lbfgs')
clf.fit(trainX,trainY)

print time.clock()-tic


31.284506
training
5.681214


In [14]:
print trainX.shape
print testX.shape

(89742, 10485)
(57356, 10485)


In [15]:
print split_index
assert not  trainX == testX
assert not  trainY == testY

223


/usr/local/lib/python2.7/site-packages/scipy/sparse/compressed.py:233: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  app.launch_new_instance()


In [17]:
ptags = clf.predict(testX)
assert not len(test_data) == len(train_data)
print len(train_data)
print len (test_data)

223
149


In [25]:

for ent in tags:
    print "ent", ent
    print "len ptags", len(ptags)
    print set(ptags)
    correct = [ptags[i] == ent and testY[i] == ent for i in range(len(ptags))]
    guessed = [ptags[i] == ent for i in range(len(ptags))]
    total   = [testY[i] == ent for i in range(len(ptags))]


    accuracy = sum(correct) *1./sum(guessed)
    recall   = sum(correct) * 1./sum(total)
    f1 = accuracy * recall * 2. / (accuracy + recall)

    print "FOR ", int2tags[ent]
    print "PERCISION", accuracy
    print "RECALL", recall
    print "F1", f1
    print "-------------"
    print "NUM CORRECT", len(correct)
    print "NUM GUESSED", len (guessed)
    print "Entities of name", len(total)
    print "_______________________"

#######
feature_list = (word_vocab.keys() + helper.other_features) * (previous_n+next_n+1)  + word_vocab.keys() + ['previous_one'] * len(tags) + ['previous_two'] * len(tags)+ ['previous_three'] * len(tags)
# getTopFeatures(clf,tags,feature_list)
if trained_model != "":
    pickle.dump([clf, previous_n,next_n, word_vocab,helper.other_features], open( trained_model, "wb" ) )


ent 0
len ptags 57356
set([0.0, 1.0, 2.0, 3.0, 4.0])
FOR  TAG
PERCISION 0.988217367423
RECALL 0.993816931273
F1 0.991009239517
-------------
NUM CORRECT 57356
NUM GUESSED 57356
Entities of name 57356
_______________________
ent 1
len ptags 57356
set([0.0, 1.0, 2.0, 3.0, 4.0])
FOR  shooterName
PERCISION 0.288343558282
RECALL 0.161512027491
F1 0.20704845815
-------------
NUM CORRECT 57356
NUM GUESSED 57356
Entities of name 57356
_______________________
ent 2
len ptags 57356
set([0.0, 1.0, 2.0, 3.0, 4.0])
FOR  killedNum
PERCISION 0.84693877551
RECALL 0.65873015873
F1 0.741071428571
-------------
NUM CORRECT 57356
NUM GUESSED 57356
Entities of name 57356
_______________________
ent 3
len ptags 57356
set([0.0, 1.0, 2.0, 3.0, 4.0])
FOR  woundedNum
PERCISION 0.661375661376
RECALL 0.581395348837
F1 0.618811881188
-------------
NUM CORRECT 57356
NUM GUESSED 57356
Entities of name 57356
_______________________
ent 4
len ptags 57356
set([0.0, 1.0, 2.0, 3.0, 4.0])
FOR  city
PERCISION 0.64453961456

NameError: name 'word_vocab' is not defined